# Differential Privacy
> Como realizar analisis estadisticos sobre datos sin comprometer la privacidad de los mismos

In [ ]:
#hide
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
#hide
from nbdev.showdoc import *

## Qué es Differential Privacy
Differential Privacy o Privacidad Diferencial busca que podamos realizar analisis estadisticos sobre datos sin violar la privacidad de las personas que puedan existir dentro de ese dataset. Primero tenemos que explicar cuando estamos conservando la privacidad de un usuario.

Podemos decir que estamos **preservando la privacidad** de un grupo de personas cuando después de un análisis, el analizador no sabe nada sobre las personas en el dataset, estos permanecen sin ser "observados".

Cynthia Dwork, pionera en el campo de privacidad diferencial, define ```Differential Privacy``` de de la siguiente manera
> Differential Privacy describe una promesa, hecha por el individuo que utilizará los datos, al individuo que provee los datos, y la promesa es:
>
>"No se verá afectado, de manera adversa o de otro modo, al permitir que sus datos sean utilizados en cualquier estudio o analysis, sin importar qué otros estudios, conjuntos de datos, o fuentes de información hay disponibles"

## Anonimizar datos no es suficiente
Si tenemos un dataset anonimizada, y otro grupo libera un dataset relacionado también anonimizado, muchas veces es posible combinar estos datasets y lograr desanonimizar los datos. Esto fue demostrado con varios ejemplos, como el uso de datasets omo IMDB para desanonimizar usuarios de MovieLens.

## Differential Privacy en una base de datos de juguete

Simulemos que tenemos una base de datos de 5000 usuarios, donde solo tenemos una columna que almacena unos y ceros. Podemos suponer que si esta base de datos fuera una sobre pacientes de cancer, cuando una fila tienen un 1 supongamos que el paciente tiene cancer y 0 cuando no lo tiene.

Si removemos una persona de las 5000 podemos estar seguros de que la información de dicho paciente no se filtró. Entonces, es posible realizar una consulta que no cambie sin importar a quien removamos de la base de datos?

## Primero creamos la Base de Datos

Inicializamos una lizta randomica de 1s y 0s donde cada entrada se corresponde directamente con el numero de personas en la base de datos

In [ ]:
import torch

# Numero de entradas en DB
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([ True, False,  True,  ...,  True,  True, False])

## Generar Based de Datos Paralelas

Clave para la definicion de ```Differential privacy``` es la habilidad de preguntar "Cuando consultando una base de datos, si remuevo los datos de una base de datos, será la salida de la consulta diferente?" Para poder probar esto, debemos construir lo que llamamos "bases de datos paralelas", que son bases de datos simples con una entrada removida.

En este primer ejemplo, crearemos una lista de todas las bases de datos paralelas a la que actualmente está contenida en la variable ```db```. Luego crearemos una funcion que""
- Creará una base de datos inicial
- Creará todas las bases de datos paralelas

In [ ]:
def get_parallel_db(db, remove_index):
    return torch.cat((db[0:remove_index], 
                      db[remove_index+1:]))

In [ ]:
get_parallel_db(db, 1).shape

torch.Size([4999])

In [ ]:
def get_parallel_dbs(db):
    parallel_dbs = []
    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
        
    return parallel_dbs

In [ ]:
parallel_dbs = get_parallel_dbs(db)
len(parallel_dbs)

5000

In [ ]:
len(parallel_dbs[0])

4999

In [ ]:
def create_db_and_parallels(num_entries):
    db = torch.rand(num_entries) > 0.5
    parallel_dbs = get_parallel_dbs(db)
    return (db, parallel_dbs)

In [ ]:
db, pdbs = create_db_and_parallels(10)

In [ ]:
db

tensor([False,  True,  True,  True, False,  True,  True, False,  True, False])

In [ ]:
pdbs

[tensor([ True,  True,  True, False,  True,  True, False,  True, False]),
 tensor([False,  True,  True, False,  True,  True, False,  True, False]),
 tensor([False,  True,  True, False,  True,  True, False,  True, False]),
 tensor([False,  True,  True, False,  True,  True, False,  True, False]),
 tensor([False,  True,  True,  True,  True,  True, False,  True, False]),
 tensor([False,  True,  True,  True, False,  True, False,  True, False]),
 tensor([False,  True,  True,  True, False,  True, False,  True, False]),
 tensor([False,  True,  True,  True, False,  True,  True,  True, False]),
 tensor([False,  True,  True,  True, False,  True,  True, False, False]),
 tensor([False,  True,  True,  True, False,  True,  True, False,  True])]

# Midiendo la privacidad de una consulta
Queremos ser capaces de consultar nuestra base de datos y evaluar si los resultados de una consulta está filtrando informacion privada. Esto se trata de evaluar si la salida de una consulta cambia cuando removemos a alguien de la base de datos. Específicamente queremos evaluar la máxima cantidad en la que una consulta cambia cuando alguien es removido (maximo sobre todas las posibles personas que pueden ser removidas. Para poder evaluar cuanta "privacidad se filtra" o pierde, vamos a iterar sobre cada persona en la base de datos y medir la diferencia entre la salida de la consulta relativa a cuando consultamos la base de datos completa.

A modo de ejemplo, acamos que nuestra primera "consulta" sea una suma, es decir, vamos a contar la cantidad de 1s que hay en la base de datos.

In [ ]:
db, pdbs = create_db_and_parallels(5000)

In [ ]:
def query(db):
    return db.sum()

In [ ]:
full_db_result = query(db)

In [ ]:
sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    db_distance = torch.abs(pdb_result - full_db_result)
    if (db_distance > sensitivity):
        sensitivity = db_distance

In [ ]:
sensitivity

tensor(1)

## Sensitivity o L1 Sensitivity
Es el valor maximo en el que una consulta cambia cuando removemos un individuo del dataset

## Definamos una función para medir la sensibilidad de cualquier funcion

In [ ]:
def sensitivity(query, n_entries=1000):
    db, pdbs = create_db_and_parallels(n_entries)
    sensitivity = 0
    full_db_result = query(db)
    for pdb in pdbs:
        pdb_result = query(pdb)
        db_distance = torch.abs(pdb_result - full_db_result)
        if (db_distance > sensitivity):
            sensitivity = db_distance
    
    return sensitivity

In [ ]:
l1 = sensitivity(query, 5000)

In [ ]:
l1

tensor(1)

Evaluemos la sensibilidad de la funcion de promedio

In [ ]:
def mean(db):
    return db.float().mean()

In [ ]:
sensitivity(mean, 1000)

tensor(0.0005)

## Calculando la L1 Sensitivity de la función Threshold

In [ ]:
def threshold_func(db, threshodl=5):
    return (db.sum() > threshodl).float()

In [ ]:
sensitivity(threshold_func, 10)

0

In [ ]:
for i in range(10):
    sen_f = sensitivity(threshold_func, n_entries=10)
    print(sen_f)

0
0
0
0
0
0
tensor(1.)
tensor(1.)
0
tensor(1.)


## Differencing Attack

In [ ]:
db, _ = create_db_and_parallels(num_entries=100)
pdb = get_parallel_db(db, 10)

In [ ]:
full_db = query(db)
tenth = query(pdb)
value = torch.abs(tenth - full_db)

In [ ]:
value

tensor(0)

In [ ]:
db[10]

tensor(False)